In [1]:
%cd ../src
%load_ext autoreload
%autoreload 2

/Users/tccuong1404/Documents/Project/lawyer-assist/src


/Users/tccuong1404/Library/Caches/pypoetry/virtualenvs/lawyer-assist-hoWIZQcw-py3.12/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from util import config

/Users/tccuong1404/Library/Caches/pypoetry/virtualenvs/lawyer-assist-hoWIZQcw-py3.12/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
from glob import glob
paths = glob(config.DATA_QNA_DIR + "/*")
print(paths[0])

/Users/tccuong1404/Documents/Project/lawyer-assist/data/qna/e77dd4a4c79cc8337dcead263fa4cc23.json


In [4]:
from tqdm import tqdm
import pandas as pd
import json
res = [] 
for x in tqdm(paths):
    data = json.load(open(x))["data"]
    for t in data:
        if not ("question" in t and "answer" in t):
            continue
        
        if t["question"] and t["answer"]:
            res.append(t)

100%|██████████| 1551/1551 [00:00<00:00, 7595.64it/s]


In [7]:
from api.models import GemmaModel_MLX

model = GemmaModel_MLX()
model.generate("Ngân hàng có cung cấp thông tin chuyển tiền")

Fetching 6 files: 100%|██████████| 6/6 [00:00<00:00, 110862.66it/s]
2024-08-31 07:41:28.518 | INFO     | api.models._gemma:__init__:20 - Model loaded: tcstrength/gemma-2b-lawyer-assist


<generator object stream_generate at 0x575bff6e0>

In [13]:
for item in tqdm(res):
    resp = model.generate(item["question"])
    resp = "".join([a for a in resp])
    item["finetune"] = resp

100%|██████████| 2149/2149 [13:40:48<00:00, 22.92s/it]  


In [15]:
df = pd.DataFrame(res)

In [17]:
df = df.drop(columns=["reference"])

In [18]:
config.DATA_QNA_DIR

'/Users/tccuong1404/Documents/Project/lawyer-assist/data/qna'

In [19]:
df.to_parquet("/Users/tccuong1404/Documents/Project/lawyer-assist/data/data.parquet")